In [42]:
import pandas as pd

#Import data (note: fire shapefiles used is progression from NRC)
intersection_df = pd.read_csv(r'C:\Users\selamin\Documents\intersectionallprovinces2.csv')
census_df = pd.read_csv(r'C:\Users\selamin\Documents\Shapefiles\censusbyDA.csv')

#View imported tables
intersection_df.head()
census_df.head()

Total Population affected: 199610.0
Total Population affected by Province:
    PRUID                   Province  Affected population count
8      59           British Columbia                    95659.0
7      48                    Alberta                    45805.0
6      47               Saskatchewan                    16310.0
5      46                   Manitoba                    11303.0
1      12                Nova Scotia                     8322.0
3      24                     Quebec                     7190.0
10     61      Northwest Territories                     5685.0
4      35                    Ontario                     5554.0
9      60                      Yukon                     2322.0
0      10  Newfoundland and Labrador                      786.0
2      13              New Brunswick                      656.0
11     62                    Nunavut                       18.0


In [49]:
#Convert data types for uniformity
intersection_df['DAUID'] = intersection_df['DAUID'].astype(str)
census_df['censusDAUID'] = census_df['censusDAUID'].astype(str)
# Convert the 'Population' column to numeric
census_df['Population'] = pd.to_numeric(census_df['Population'], errors='coerce')

#Merge the two dfs
merged_df = intersection_df.merge(census_df, left_on='DAUID', right_on='censusDAUID', how='left')

#Filter out rows with no match
merged_df = merged_df.dropna(subset=['Population'])

#Keep the first match and drop duplicates
merged_df = merged_df.drop_duplicates(subset='DAUID', keep='first')

# Define a dictionary to map PRUID values to province names
pruid_to_province = {
    48: 'Alberta',
    59: 'British Columbia',
    46: 'Manitoba',
    13: 'New Brunswick',
    10: 'Newfoundland and Labrador',
    61: 'Northwest Territories',
    12: 'Nova Scotia',
    62: 'Nunavut',
    35: 'Ontario',
    11: 'Prince Edward Island',
    24: 'Quebec',
    47: 'Saskatchewan',
    60: 'Yukon'
}
# Create a new column 'Province' by mapping 'PRUID' values
merged_df['Province'] = merged_df['PRUID'].map(pruid_to_province)

#View final table
merged_df.head()
merged_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 424 entries, 0 to 14444
Data columns (total 23 columns):
 #   Column                                                                                       Non-Null Count  Dtype  
---  ------                                                                                       --------------  -----  
 0   DAUID                                                                                        424 non-null    object 
 1   DGUID_x                                                                                      424 non-null    object 
 2   LANDAREA                                                                                     424 non-null    float64
 3   PRUID                                                                                        424 non-null    int64  
 4   CLUSTERID                                                                                    424 non-null    int64  
 5   DATE                              

In [50]:

#Export final csv
merged_df.to_csv(r'C:\Users\selamin\Documents\Population_in_fire_zone2.csv', index=False)



In [51]:
#Quick Look at Population Totals by Province

#Sum the 'Population' values for the matching rows
total_population = merged_df['Population'].sum()

#Groupby province name
province_population = merged_df.groupby(['PRUID','Province'])['Population'].sum().reset_index()

# Rename the 'Population' column to 'Affected population count'
province_population.rename(columns={'Population': 'Affected population count'}, inplace=True)

# Sort the table in descending order of the 'Affected population count'
province_population = province_population.sort_values(by='Affected population count', ascending=False)

# Print the updated table
print("Total Population affected:", total_population)
print("Total Population affected by Province:")
print(province_population)


Total Population affected: 199610.0
Total Population affected by Province:
    PRUID                   Province  Affected population count
8      59           British Columbia                    95659.0
7      48                    Alberta                    45805.0
6      47               Saskatchewan                    16310.0
5      46                   Manitoba                    11303.0
1      12                Nova Scotia                     8322.0
3      24                     Quebec                     7190.0
10     61      Northwest Territories                     5685.0
4      35                    Ontario                     5554.0
9      60                      Yukon                     2322.0
0      10  Newfoundland and Labrador                      786.0
2      13              New Brunswick                      656.0
11     62                    Nunavut                       18.0
